# Launch training in notebook
In this notebook, we demonstrate how to finetune pretrained DANNCE/s-DANNCE models on new set of data.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import torch

from dannce.cli import get_parser, build_clarg_params
from dannce.interface import dannce_train, sdannce_train

cwd = os.getcwd()

In [2]:
RUNNING_MODE = "train"
MODEL_TYPE = "sdannce" # options: "dannce", "sdannce"

# where a io.yaml file with valid `exp` must reside
# check out notebook 1 for more details if this is unclear
EXPROOT = '../demo/2021_07_06_M3_M6'

# training configurations
CONFIG = '../configs/sdannce_rat_config.yaml' # '../configs/dannce_rat_config.yaml' for dannce training

TRAIN_MODE = "finetune" # options: "new", "finetune"
FINETUNE_WEIGHTS = "../demo/weights/SDANNCE_gcn_bsl_FM_ep100.pth" # only applied if TRAIN_MODE == "finetune", can be either DANNCE/s-DANNCE weights
EPOCHS = 2 # only for demo purposes here

DANNCE_TRAIN_DIR = "SDANNCE/train_from_notebook"

In [15]:
def launch_train(
    exproot=EXPROOT,
    config=CONFIG,
    model_type=MODEL_TYPE,
    train_mode=TRAIN_MODE,
    net_type="compressed_dannce",
    finetune_weights=FINETUNE_WEIGHTS,
    dannce_train_dir=DANNCE_TRAIN_DIR,
    epochs=EPOCHS,
    use_npy=True, # save the generated image volumes for future use
):
    exproot = os.path.join(cwd, exproot)
    config = os.path.join(cwd, config)
    finetune_weights = os.path.join(cwd, finetune_weights)

    # parameter arguments override
    # check out possible configuration parameters in the config
    # and/or `dannce/__init__.py`
    arguments = {
        "dannce-finetune-weights": finetune_weights,
        "dannce-train-dir": dannce_train_dir,
        "train-mode": train_mode,
        "net-type": net_type,
        # "batch-size": 2,
        "use-npy": use_npy,
        "epochs": epochs,
    }

    # DANNCE must run within the experiment directory
    os.chdir(exproot)

    # compose the DANNCE command
    cmds = ['dannce', RUNNING_MODE, model_type, config]
    # override default arguments if specified
    for k, v in arguments.items():
        cmds += [f"--{k}", str(v)]

    # set arguments and launch command
    sys.argv = cmds
    args = get_parser()
    params = build_clarg_params(
        args,
        dannce_net=(args.mode == "dannce") | (args.mode == "sdannce"),
        prediction=(args.command == "predict"),
    )
    
    if model_type == "dannce":
        train_fcn = dannce_train
    elif model_type == "sdannce":
        train_fcn = sdannce_train
    else:
        raise ValueError(f"Invalid model type {model_type}. Choose 'dannce' or 'sdannce'.")
    train_fcn(params)

    # please manually clear the CUDA cache to avoid OOM
    torch.cuda.empty_cache()

In [16]:
launch_train()

2025-02-27 16:31:23.668 | WARNING  | dannce.config:inherit_config:307 - io_config not found in io.yaml file, falling back to default
2025-02-27 16:31:23.670 | WARNING  | dannce.config:inherit_config:307 - camnames not found in io.yaml file, falling back to default
2025-02-27 16:31:23.670 | WARNING  | dannce.config:inherit_config:307 - n_instances not found in io.yaml file, falling back to default
2025-02-27 16:31:23.671 | WARNING  | dannce.config:inherit_config:307 - random_seed not found in io.yaml file, falling back to default
2025-02-27 16:31:23.672 | WARNING  | dannce.config:inherit_config:307 - crop_height not found in io.yaml file, falling back to default
2025-02-27 16:31:23.673 | WARNING  | dannce.config:inherit_config:307 - crop_width not found in io.yaml file, falling back to default
2025-02-27 16:31:23.673 | WARNING  | dannce.config:inherit_config:307 - vmin not found in io.yaml file, falling back to default
2025-02-27 16:31:23.674 | WARNING  | dannce.config:inherit_config:30

Adjusting learning rate of group 0 to 1.0000e-04.


Epoch[1/2] train_BoneLengthLoss: 0.0208 train_ConsistencyLoss: 0.1548 train_L1DiffLoss: 0.7996 : 100%|██████████| 412/412 [08:34<00:00,  1.25s/it] 


Adjusting learning rate of group 0 to 1.0000e-04.


100%|██████████| 16/16 [00:02<00:00,  5.37it/s]
2025-02-27 16:40:05.410 | INFO     | dannce.engine.trainer.dannce_trainer:train:88 - Epoch[1/2]
train_BoneLengthLoss: 0.0278
train_ConsistencyLoss: 0.2176
train_L1DiffLoss: 4.7060
train_euclidean_distance_3D: 29.3084
val_BoneLengthLoss: 0.0274
val_ConsistencyLoss: 0.0000
val_L1DiffLoss: 1.1611
val_euclidean_distance_3D: 7.1603

Epoch[2/2] train_BoneLengthLoss: 0.0227 train_ConsistencyLoss: 0.2508 train_L1DiffLoss: 19.9912 : 100%|██████████| 412/412 [08:36<00:00,  1.25s/it]


Adjusting learning rate of group 0 to 1.0000e-04.


100%|██████████| 16/16 [00:02<00:00,  5.89it/s]
2025-02-27 16:48:44.719 | INFO     | dannce.engine.trainer.dannce_trainer:train:88 - Epoch[2/2]
train_BoneLengthLoss: 0.0291
train_ConsistencyLoss: 0.2104
train_L1DiffLoss: 4.4322
train_euclidean_distance_3D: 27.7869
val_BoneLengthLoss: 0.0253
val_ConsistencyLoss: 0.0000
val_L1DiffLoss: 1.2240
val_euclidean_distance_3D: 7.5768

2025-02-27 16:48:44.721 | INFO     | dannce.engine.trainer.base_trainer:_save_checkpoint:72 - Saving checkpoint: SDANNCE/train_from_notebook/checkpoint-epoch2.pth ...


In [18]:
# list out items in the experiment directory
os.chdir(cwd)
os.chdir(EXPROOT)
os.chdir(DANNCE_TRAIN_DIR)

print("Experiment directory contents:")
print(os.listdir())

Experiment directory contents:
['training.csv', 'checkpoint.pth', 'checkpoint-epoch2.pth', 'logs', 'val_samples.pickle', 'stats_dannce_train.log', 'train_samples.pickle', 'params.yaml', 'params.pickle']
